# Este cogumelo é venenoso?

Este kernel é minha tentativa de aprender o básico do Machine Learning. Usaremos o modelo da Árvore de Decisão, porque é mais fácil ver e entender.

Use ou defina o conjunto de dados de classificação para determinar se um cogumelo encontrado é venenoso ou comestível

### Index 
1. Importando bibliotecas e dados
2. Análise e Limpeza de Dados
3. Treinando um modelo
4. Conclusão


## 1. Importando bibliotecas e dados

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualisation
sns.set(style="darkgrid")
import matplotlib.pyplot as plt # data plotting

import warnings
warnings.simplefilter("ignore") # Remove certain warnings from Machine Learning Models


In [ ]:
import pandas as pd
data = pd.read_csv("../input/mushroom-classification/mushrooms.csv")
data.head(2)

## 2. Análise e Limpeza de Dados
Before we can train a model to help us determine if a mushroom is poisonous we have to look at what kind of data we have. After that we'll clean the data so it can be used by a model.


In [ ]:
data.describe()

Temos muitas colunas diferentes. Vamos tentar entender melhor o efeito que eles têm na classe de um cogumelo, fazendo alguns gráficos.

Vamos começar analisando a distribuição de cogumelos comestíveis e venenosos em nossos dados. Como você pode ver, a distribuição é quase 50/50, o que é muito bom quando você permite que um modelo de aprendizado de máquina use seus dados. Isso significa que não precisará se esforçar para encontrar correlações.

In [ ]:
sns.countplot(data['class'])

A maioria das colunas relacionadas ao limite parece bastante equilibrada entre comestível e venenosa.

Notáveis são a forma de tampa **k** *(cap-shape)*, a superfície da tampa **f** *(cap-surface)* e a cor da tampa **w** *(cap-color)* por não terem uma distribuição razoavelmente igual.

In [ ]:
fig, ax =plt.subplots(1,3, figsize=(15,5))
sns.countplot(x="cap-shape", hue='class', data=data, ax=ax[0])
sns.countplot(x="cap-surface", hue='class', data=data, ax=ax[1])
sns.countplot(x="cap-color", hue='class', data=data, ax=ax[2])
fig.tight_layout()
fig.show()

Os hematomas *(bruises)* são uma indicação muito melhor se um cogumelo é venenoso ou não. Você pode ver claramente a distribuição no gráfico. Os odores são ainda mais claros, apenas o odor **n** é venenoso e comestível. Mesmo assim, não é realmente uma distribuição justa, comestível sendo várias vezes maior. Ambas são boas colunas para determinar a classe de um cogumelo.


In [ ]:
fig, ax =plt.subplots(1,2, figsize=(15,5))
sns.countplot(x="bruises", hue='class', data=data, ax=ax[0])
sns.countplot(x="odor", hue='class', data=data, ax=ax[1])
fig.tight_layout()
fig.show()

As brânquias são distribuídas de maneira um pouco mais uniforme novamente, embora existam alguns desvios no espaçamento, tamanho e cor

In [ ]:
fig, ax =plt.subplots(1,4, figsize=(20,5))
sns.countplot(x="gill-attachment", hue='class', data=data, ax=ax[0])
sns.countplot(x="gill-spacing", hue='class', data=data, ax=ax[1])
sns.countplot(x="gill-size", hue='class', data=data, ax=ax[2])
sns.countplot(x="gill-color", hue='class', data=data, ax=ax[3])
fig.tight_layout()
fig.show()

Novamente, as colunas de caule *(stalk)* são distribuídas de maneira bastante uniforme. As colunas de **superfície** e **cor** que podem ser úteis para classificação.

In [ ]:
fig, ax =plt.subplots(2,3, figsize=(20,10))
sns.countplot(x="stalk-shape", hue='class', data=data, ax=ax[0,0])
sns.countplot(x="stalk-root", hue='class', data=data, ax=ax[0,1])
sns.countplot(x="stalk-surface-above-ring", hue='class', data=data, ax=ax[0,2])
sns.countplot(x="stalk-surface-below-ring", hue='class', data=data, ax=ax[1,0])
sns.countplot(x="stalk-color-above-ring", hue='class', data=data, ax=ax[1,1])
sns.countplot(x="stalk-color-below-ring", hue='class', data=data, ax=ax[1,2])
fig.tight_layout()
fig.show()

A maioria dessas colunas não é muito interessante, mas o **tipo de anel** *(ring-type)* possui algumas informações úteis.

In [ ]:
fig, ax =plt.subplots(2,2, figsize=(15,10))
sns.countplot(x="veil-type", hue='class', data=data, ax=ax[0,0])
sns.countplot(x="veil-color", hue='class', data=data, ax=ax[0,1])
sns.countplot(x="ring-number", hue='class', data=data, ax=ax[1,0])
sns.countplot(x="ring-type", hue='class', data=data, ax=ax[1,1])
fig.tight_layout()
fig.show()

Essas últimas colunas têm alguns valores discrepantes mais fortes que podemos usar para previsão. especialmente **cor de impressão de esporos** (spore-print-color).

In [ ]:
fig, ax =plt.subplots(1,3, figsize=(20,5))
sns.countplot(x="spore-print-color", hue='class', data=data, ax=ax[0])
sns.countplot(x="population", hue='class', data=data, ax=ax[1])
sns.countplot(x="habitat", hue='class', data=data, ax=ax[2])
fig.tight_layout()
fig.show()

Agora que sabemos como são nossos dados, podemos limpá-los. Vamos começar transformando as colunas com apenas 2 valores diferentes em booleanos.

In [ ]:
# Make column class True/False for isPoisonous
data['class'].replace('p', 1, inplace = True)
data['class'].replace('e', 0, inplace = True)

# Bruises: t = True / f = False
data['bruises'].replace('t', 1, inplace = True)
data['bruises'].replace('f', 0, inplace = True)

In [ ]:
data = pd.get_dummies(data)

pd.set_option("display.max_columns",200)
data.head(5)

Vamos fazer listas com as colunas para que possamos fazer alguns mapas de calor de correlação.

In [ ]:
Target = ['class']
bruisesColumn = ['bruises']
capColumns = list(data.columns[2:22])
odorColumns = list(data.columns[22:31])
gillColumns = list(data.columns[31:49])
stalkColumns = list(data.columns[49:82])
veilColumns = list(data.columns[82:87])
ringColumns = list(data.columns[87:95])
sporeColumns = list(data.columns[95:104])
populationColumns = list(data.columns[104: 110])
habitatColumns = list(data.columns[110:117])

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(data[Target+odorColumns].corr(), annot=True)

In [ ]:
plt.subplots(figsize=(10,10))
sns.heatmap(data[Target+populationColumns].corr(), annot=True)

## 3. Treinando um modelo

Agora que temos dados utilizáveis, podemos começar a treinar nossos dados. Usaremos o modelo da Árvore de Decisão, pois é mais fácil de visualizar e entender.

Primeiro, precisamos criar X e Y DataFrames. Os DataFrames Y contêm os dados que previmos, os X DataFrames contêm os dados que o modelo usa para prever y.

In [ ]:
#Create X & y
X = data.iloc[:, 1:]
y = data['class']

Para testar nosso modelo, dividimos os dados em dados de treinamento e teste. Isso garante que o modelo não memorize apenas os dados em vez de encontrar correlações.

In [ ]:
#Create Testing and Training Data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Finalmente, podemos importar e testar nossa Árvore de Decisão. Estamos usando uma versão do classificador que nos fornece uma variável booleana (0 ou 1). Se tivéssemos usado um Regressor, teríamos obtido um número entre 0 e 1.

Para simplificar, limitaremos a árvore a uma profundidade máxima de 5. Em seguida, ajustaremos os dados de treinamento. O ajuste prepara o modelo para o trabalho real, neste caso a identificação do X_test.

In [ ]:
from sklearn import tree

dtc = tree.DecisionTreeClassifier(max_depth=2, random_state=0)
dtc.fit(X_train, y_train)

dtc.score(X_test, y_test)


Nossa pontuação final é 0,954, o que é muito bom! Isso significa que estamos corretos em 95,4% do tempo. Para aumentar isso, podemos remover o limite max_depth para que a árvore possa crescer maior.

Agora vamos ver como é a nossa Árvore de Decisão! A árvore é construída com folhas e essas folhas contêm informações sobre o nosso modelo:
* Condição da folha
* Gini (ou chance de medição incorreta de uma amostra aleatória de treinamento naquele ponto)
* O número de amostras que passaram durante o ajuste
* Classe (ou previsão) da amostra nesse ponto

In [ ]:
import graphviz
dot_data = tree.export_graphviz(dtc, feature_names=X.columns.values, class_names=['Edible', 'Poisonous'], filled=True )
graphviz.Source(dot_data) 

## Conclusão

Criamos um modelo com 95% de acurácia para predizer se um cogumelo é comestível ou venenoso.